In [27]:
# cd to the root directory
!python setup.py install
!qiime dev refresh-cache

running install
/opt/conda/envs/qiime2-2022.2/lib/python3.8/site-packages/setuptools/command/install.py:34: SetuptoolsDeprecationWarning: setup.py install is deprecated. Use build and pip and other standards-based tools.
  warnings.warn(
/opt/conda/envs/qiime2-2022.2/lib/python3.8/site-packages/setuptools/command/easy_install.py:156: EasyInstallDeprecationWarning: easy_install command is deprecated. Use build and pip and other standards-based tools.
  warnings.warn(
running bdist_egg
running egg_info
writing q2_gglasso.egg-info/PKG-INFO
writing dependency_links to q2_gglasso.egg-info/dependency_links.txt
writing entry points to q2_gglasso.egg-info/entry_points.txt
writing top-level names to q2_gglasso.egg-info/top_level.txt
reading manifest file 'q2_gglasso.egg-info/SOURCES.txt'
adding license file 'LICENSE'
writing manifest file 'q2_gglasso.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
running install_lib
running build_py
creating build/bdist.linux-x86_

In [28]:
import pandas as pd
import numpy as np
import qiime2 as q2


import matplotlib.pyplot as plt
from scipy import stats

from q2_gglasso.utils import PCA, remove_biom_header

In [31]:
!qiime tools import \
   --type EMPPairedEndSequences \
   --input-path example/data/atacama/emp-paired-end-sequences \
   --output-path example/data/atacama/emp-paired-end-sequences.qza

['/opt/conda/envs/qiime2-2022.2/bin', '/opt/conda/envs/qiime2-2022.2/lib/python38.zip', '/opt/conda/envs/qiime2-2022.2/lib/python3.8', '/opt/conda/envs/qiime2-2022.2/lib/python3.8/lib-dynload', '/opt/conda/envs/qiime2-2022.2/lib/python3.8/site-packages', '/opt/conda/envs/qiime2-2022.2/lib/python3.8/site-packages/q2_gglasso-0.0.0.dev0-py3.8.egg', '/opt/project']
Imported example/data/atacama/emp-paired-end-sequences as EMPPairedEndDirFmt to example/data/atacama/emp-paired-end-sequences.qza


In [32]:
!qiime demux emp-paired \
  --m-barcodes-file example/data/atacama/sample-metadata.tsv \
  --m-barcodes-column barcode-sequence \
  --p-rev-comp-mapping-barcodes \
  --i-seqs example/data/atacama/emp-paired-end-sequences.qza \
  --o-per-sample-sequences example/data/atacama/demux-full.qza \
  --o-error-correction-details example/data/atacama/demux-details.qza

['/opt/conda/envs/qiime2-2022.2/bin', '/opt/conda/envs/qiime2-2022.2/lib/python38.zip', '/opt/conda/envs/qiime2-2022.2/lib/python3.8', '/opt/conda/envs/qiime2-2022.2/lib/python3.8/lib-dynload', '/opt/conda/envs/qiime2-2022.2/lib/python3.8/site-packages', '/opt/conda/envs/qiime2-2022.2/lib/python3.8/site-packages/q2_gglasso-0.0.0.dev0-py3.8.egg', '/opt/project']
Saved SampleData[PairedEndSequencesWithQuality] to: example/data/atacama/demux-full.qza
Saved ErrorCorrectionDetails to: example/data/atacama/demux-details.qza


In [33]:
!qiime demux summarize \
  --i-data example/data/atacama/demux-full.qza \
  --o-visualization example/data/atacama/demux-full.qzv

['/opt/conda/envs/qiime2-2022.2/bin', '/opt/conda/envs/qiime2-2022.2/lib/python38.zip', '/opt/conda/envs/qiime2-2022.2/lib/python3.8', '/opt/conda/envs/qiime2-2022.2/lib/python3.8/lib-dynload', '/opt/conda/envs/qiime2-2022.2/lib/python3.8/site-packages', '/opt/conda/envs/qiime2-2022.2/lib/python3.8/site-packages/q2_gglasso-0.0.0.dev0-py3.8.egg', '/opt/project']
Saved Visualization to: example/data/atacama/demux-full.qzv


In [34]:
!qiime tools export \
  --input-path example/data/atacama/demux-full.qzv \
  --output-path example/data/atacama/demux-full/

Exported example/data/atacama/demux-full.qzv as Visualization to directory example/data/atacama/demux-full/


In [35]:
!qiime demux filter-samples \
  --i-demux example/data/atacama/demux-full.qza \
  --m-metadata-file example/data/atacama/demux-full/per-sample-fastq-counts.tsv \
  --p-where 'CAST([forward sequence count] AS INT) > 100' \
  --o-filtered-demux example/data/atacama/demux.qza

['/opt/conda/envs/qiime2-2022.2/bin', '/opt/conda/envs/qiime2-2022.2/lib/python38.zip', '/opt/conda/envs/qiime2-2022.2/lib/python3.8', '/opt/conda/envs/qiime2-2022.2/lib/python3.8/lib-dynload', '/opt/conda/envs/qiime2-2022.2/lib/python3.8/site-packages', '/opt/conda/envs/qiime2-2022.2/lib/python3.8/site-packages/q2_gglasso-0.0.0.dev0-py3.8.egg', '/opt/project']
Saved SampleData[PairedEndSequencesWithQuality] to: example/data/atacama/demux.qza


In [36]:
!qiime dada2 denoise-paired \
  --i-demultiplexed-seqs example/data/atacama/demux.qza \
  --p-trim-left-f 13 \
  --p-trim-left-r 13 \
  --p-trunc-len-f 150 \
  --p-trunc-len-r 150 \
  --o-table example/data/atacama/table.qza \
  --o-representative-sequences example/data/atacama/rep-seqs.qza \
  --o-denoising-stats example/data/atacama/denoising-stats.qza

['/opt/conda/envs/qiime2-2022.2/bin', '/opt/conda/envs/qiime2-2022.2/lib/python38.zip', '/opt/conda/envs/qiime2-2022.2/lib/python3.8', '/opt/conda/envs/qiime2-2022.2/lib/python3.8/lib-dynload', '/opt/conda/envs/qiime2-2022.2/lib/python3.8/site-packages', '/opt/conda/envs/qiime2-2022.2/lib/python3.8/site-packages/q2_gglasso-0.0.0.dev0-py3.8.egg', '/opt/project']
Saved FeatureTable[Frequency] to: example/data/atacama/table.qza
Saved FeatureData[Sequence] to: example/data/atacama/rep-seqs.qza
Saved SampleData[DADA2Stats] to: example/data/atacama/denoising-stats.qza


In [37]:
!qiime feature-table summarize \
  --i-table example/data/atacama/table.qza \
  --o-visualization example/data/atacama/table.qzv \
  --m-sample-metadata-file example/data/atacama/sample-metadata.tsv

['/opt/conda/envs/qiime2-2022.2/bin', '/opt/conda/envs/qiime2-2022.2/lib/python38.zip', '/opt/conda/envs/qiime2-2022.2/lib/python3.8', '/opt/conda/envs/qiime2-2022.2/lib/python3.8/lib-dynload', '/opt/conda/envs/qiime2-2022.2/lib/python3.8/site-packages', '/opt/conda/envs/qiime2-2022.2/lib/python3.8/site-packages/q2_gglasso-0.0.0.dev0-py3.8.egg', '/opt/project']
Saved Visualization to: example/data/atacama/table.qzv


In [38]:
!qiime feature-table tabulate-seqs \
  --i-data example/data/atacama/rep-seqs.qza \
  --o-visualization example/data/atacama/rep-seqs.qzv

['/opt/conda/envs/qiime2-2022.2/bin', '/opt/conda/envs/qiime2-2022.2/lib/python38.zip', '/opt/conda/envs/qiime2-2022.2/lib/python3.8', '/opt/conda/envs/qiime2-2022.2/lib/python3.8/lib-dynload', '/opt/conda/envs/qiime2-2022.2/lib/python3.8/site-packages', '/opt/conda/envs/qiime2-2022.2/lib/python3.8/site-packages/q2_gglasso-0.0.0.dev0-py3.8.egg', '/opt/project']
Saved Visualization to: example/data/atacama/rep-seqs.qzv


In [39]:
!qiime metadata tabulate \
  --m-input-file example/data/atacama/denoising-stats.qza \
  --o-visualization example/data/atacama/denoising-stats.qzv

['/opt/conda/envs/qiime2-2022.2/bin', '/opt/conda/envs/qiime2-2022.2/lib/python38.zip', '/opt/conda/envs/qiime2-2022.2/lib/python3.8', '/opt/conda/envs/qiime2-2022.2/lib/python3.8/lib-dynload', '/opt/conda/envs/qiime2-2022.2/lib/python3.8/site-packages', '/opt/conda/envs/qiime2-2022.2/lib/python3.8/site-packages/q2_gglasso-0.0.0.dev0-py3.8.egg', '/opt/project']
Saved Visualization to: example/data/atacama/denoising-stats.qzv


In [40]:
!qiime composition add-pseudocount \
                    --i-table example/data/atacama/table.qza \
                    --p-pseudocount 1 \
                    --o-composition-table example/data/atacama/table_composition.qza

['/opt/conda/envs/qiime2-2022.2/bin', '/opt/conda/envs/qiime2-2022.2/lib/python38.zip', '/opt/conda/envs/qiime2-2022.2/lib/python3.8', '/opt/conda/envs/qiime2-2022.2/lib/python3.8/lib-dynload', '/opt/conda/envs/qiime2-2022.2/lib/python3.8/site-packages', '/opt/conda/envs/qiime2-2022.2/lib/python3.8/site-packages/q2_gglasso-0.0.0.dev0-py3.8.egg', '/opt/project']
Saved FeatureTable[Composition] to: example/data/atacama/table_composition.qza


In [41]:
!qiime gglasso transform-features \
     --p-transformation clr \
     --i-table example/data/atacama/table_composition.qza \
     --o-transformed-table example/data/atacama/table_clr.qza

['/opt/conda/envs/qiime2-2022.2/bin', '/opt/conda/envs/qiime2-2022.2/lib/python38.zip', '/opt/conda/envs/qiime2-2022.2/lib/python3.8', '/opt/conda/envs/qiime2-2022.2/lib/python3.8/lib-dynload', '/opt/conda/envs/qiime2-2022.2/lib/python3.8/site-packages', '/opt/conda/envs/qiime2-2022.2/lib/python3.8/site-packages/q2_gglasso-0.0.0.dev0-py3.8.egg', '/opt/project']
Saved FeatureTable[Composition] to: example/data/atacama/table_clr.qza


In [42]:
!qiime gglasso calculate-covariance \
     --p-method scaled \
     --i-table example/data/atacama/table_clr.qza \
     --o-covariance-matrix example/data/atacama/table_corr.qza

['/opt/conda/envs/qiime2-2022.2/bin', '/opt/conda/envs/qiime2-2022.2/lib/python38.zip', '/opt/conda/envs/qiime2-2022.2/lib/python3.8', '/opt/conda/envs/qiime2-2022.2/lib/python3.8/lib-dynload', '/opt/conda/envs/qiime2-2022.2/lib/python3.8/site-packages', '/opt/conda/envs/qiime2-2022.2/lib/python3.8/site-packages/q2_gglasso-0.0.0.dev0-py3.8.egg', '/opt/project']
Saved PairwiseFeatureData to: example/data/atacama/table_corr.qza


In [ ]:
!qiime gglasso solve-problem \
     --p-lambda1 0.22758 \
     --p-latent True \
     --p-mu1 6.6 \
     --i-covariance-matrix example/data/atacama/table_corr.qza \
     --o-solution example/data/atacama/solution_low.qza \
     --verbose

### Export

In [ ]:
!qiime tools export \
  --input-path example/data/atacama/table_clr.qza \
  --output-path example/data/atacama/test/table_clr

In [ ]:
!qiime tools export \
  --input-path example/data/atacama/table_corr.qza \
  --output-path example/data/atacama/test/table_corr

In [ ]:
!biom convert -i example/data/atacama/test/table_clr/feature-table.biom -o example/data/atacama/test/table_clr/clr_feature-table.tsv --to-tsv

In [ ]:
remove_biom_header(file_path="example/data/atacama/test/table_clr/clr_feature-table.tsv")